![image.png](https://i.imgur.com/a3uAqnb.png)

## From Self-Attention to Transformers: Architecture & Intuition

Now that we understand **Self-Attention**, we're ready to see how it's used to build one of the most powerful architectures in deep learning:

# ⚙️ Transformers

---

## 🧩 What Is a Transformer?

A **Transformer** is a neural network architecture built entirely using **self-attention layers**, without recurrence or convolution.

> Introduced in the paper:  
> *"Attention is All You Need"* (Vaswani et al., 2017)

Instead of processing tokens **step-by-step** (like RNNs), Transformers allow **parallel processing** of entire sequences using self-attention.

---

## 🧱 Transformer Architecture Overview

A **Transformer** is composed of:

- 🔄 **Multi-Head Self-Attention** (each head learns a different attention pattern)
- 🔧 **Feedforward Neural Network** (applied after attention)
- ➕ **Add & LayerNorm** (residual connection and normalization)
- 📐 **Positional Encoding** (since attention is order-agnostic)

Each block looks like this:

![1*vrSX_Ku3EmGPyqF_E-2_Vg.webp](https://i.imgur.com/2toEGWJ.png)

# 🌟 BERT: Bidirectional Encoder Representations from Transformers

## 📘 Introduction
**BERT** is a language representation model developed by Google in 2018. Unlike traditional models that read text either left-to-right or right-to-left, BERT reads in **both directions simultaneously** using the Transformer architecture.

> 🧠 BERT is pre-trained on a large text corpus and can be fine-tuned for a variety of downstream NLP tasks (e.g., classification, QA, NER).

---

## 🌍 Motivation

Before BERT, many NLP models processed language in a **unidirectional** way — either left-to-right or right-to-left. This limited their ability to understand the **full context** of a word within a sentence.

> 🔑 **BERT was designed to deeply understand language context by reading in both directions — simultaneously.**

### 🔁 Why Bidirectional?
- Words gain meaning from surrounding words.
- Example: 
  - Sentence: "He wore a mask to the party."
  - Sentence: "He wore a mask during surgery."

In both cases, the word *mask* has different meanings. BERT looks **left and right** to understand which meaning is correct.

---

### 🧩 Key Ideas:
- **Masked Language Modeling (MLM):**  
Randomly hides some words and asks the model to guess them based on the full context.

- **Next Sentence Prediction (NSP):**  
Helps BERT understand how sentences relate to each other — useful for tasks like QA and dialogue.

---
![emb.png](https://i.imgur.com/uXZIn9Y.png)
## 📚 References

- Paper: [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding (Devlin et al., 2018)](https://arxiv.org/abs/1810.04805)
- HuggingFace Transformers: https://huggingface.co/transformers/

In [ ]:
# !pip install transformers datasets torch arabic-reshaper python-bidi
# !pip install accelerate evaluate
# !pip install bert-score

**1. Load and Explore the Dataset**

In [3]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer
)
from datasets import Dataset, load_dataset
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import arabic_reshaper
from bidi.algorithm import get_display
import os

# os.environ["WANDB_DISABLED"] = "true"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Using device: cuda
GPU: NVIDIA GeForce RTX 4070 Ti SUPER


**1. Load and Explore the Dataset**

In [7]:
# Load MLQA Arabic (smaller multilingual QA dataset)
dataset = load_dataset("facebook/mlqa", "mlqa.ar.ar", split="test")
print("MLQA Arabic dataset loaded successfully!")
print(f"Dataset size: {len(dataset)}")

print("\nSample data:")
for i in range(0,2):
    print(f"Context: {dataset[i]['context'][:100]}...")
    print(f"Question: {dataset[i]['question']}")
    print(f"Answer: {dataset[i]['answers']['text'][0]}")
    print("-" * 50)

Generating test split:   0%|          | 0/5335 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/517 [00:00<?, ? examples/s]

MLQA Arabic dataset loaded successfully!
Dataset size: 5335

Sample data:
Context: يستخدم نفس نظام تسمية المنطقة "xx" لأجزاء أخرى من مواقع التجارب في نيفادا.وتعتبر القاعدة المستطيلة ا...
Question: أي نوع من الطرق يؤدي إلى مزارع المواشي؟
Answer: الطرق الترابية
--------------------------------------------------
Context: يستخدم نفس نظام تسمية المنطقة "xx" لأجزاء أخرى من مواقع التجارب في نيفادا.وتعتبر القاعدة المستطيلة ا...
Question: إلى أين يؤدي طريق بحيرة جرووم بالنسبة للبحيرة؟
Answer: شمال شرق
--------------------------------------------------


**2-Load Pre-trained Arabic BERT Model**

In [8]:
# Load Arabic BERT model and tokenizer
model_name = "aubmindlab/bert-base-arabertv2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

print("Model and tokenizer loaded successfully!")
print(f"Model: {model_name}")
print(f"Model type: {type(model).__name__}")
print(f"Tokenizer vocab size: {tokenizer.vocab_size}")

tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model and tokenizer loaded successfully!
Model: aubmindlab/bert-base-arabertv2
Model type: BertForQuestionAnswering
Tokenizer vocab size: 64000


**3-Test BEFORE Training (Baseline Performance)**

In [9]:
def answer_question(question, context):
    """Answer a question given a context using Arabic BERT"""

    # Tokenize input
    inputs = tokenizer(
        question,
        context,
        return_tensors="pt",
        max_length=512,
        truncation=True,
        padding=True
    )

    # Move to device if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model.to(device)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get start and end logits
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Find the most likely start and end positions
    start_index = torch.argmax(start_logits, dim=1).item()
    end_index = torch.argmax(end_logits, dim=1).item()

    # Ensure end_index is not before start_index
    if end_index < start_index:
        end_index = start_index

    # Extract answer tokens
    answer_tokens = inputs["input_ids"][0][start_index:end_index+1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    # Return empty string if no meaningful answer found
    if not answer.strip():
        return "غير محدد"

    return answer.strip()

In [10]:
# Test with real examples from the ARCD dataset - BEFORE TRAINING
print("🔴 TESTING BEFORE TRAINING (Baseline Performance)")
print("=" * 60)
print("NOTE: Poor performance expected - this is aubmindlab/bert-base-arabertv2 WITHOUT QA fine-tuning")
print("=" * 60)

# Get a few test examples from a different range to avoid train/eval overlap
test_examples = dataset.select(range(30, 35))  # Use samples 30-35 for testing

correct_predictions = 0
total_predictions = 0

for i, example in enumerate(test_examples):
    context = example['context']
    question = example['question']

    # Get ground truth answer
    ground_truth = example['answers']['text'][0]

    # Get model prediction
    predicted_answer = answer_question(question, context)

    # Check if prediction matches ground truth (case-insensitive)
    is_correct = predicted_answer.strip().lower() == ground_truth.strip().lower()
    if is_correct:
        correct_predictions += 1
    total_predictions += 1

    print(f"Example {i+1}:")
    print(f"Question: {question}")
    print(f"Context snippet: {context[:150]}...")
    print(f"Predicted: '{predicted_answer}'")
    print(f"Ground Truth: '{ground_truth}'")
    print(f"Match: {'✅' if is_correct else '❌'}")
    print("-" * 50)

# Calculate and display accuracy
baseline_accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\n🔴 PRE-TRAINING Accuracy: {correct_predictions}/{total_predictions} ({baseline_accuracy:.1f}%)")
print("This baseline will improve significantly after fine-tuning on ARCD!")

🔴 TESTING BEFORE TRAINING (Baseline Performance)
NOTE: Poor performance expected - this is aubmindlab/bert-base-arabertv2 WITHOUT QA fine-tuning
Example 1:
Question: من دعم الرأي العام؟
Context snippet: ومع ذلك، ظل العامة يتعاطفون مع الملكة كاثرين. وفي ليلة من خريف عام 1531، بينما كانت آن تتناول طعامها في منزل ريفي على نهر التايمز، هاجمها حشد نسائي غا...
Predicted: '##لكة كاثرين. وفي ليلة من خريف عام 1531 ، بينما كانت آن تتناول طعامها في منزل ريفي على نهر التايمز ، هاجمها حشد نسائي غاضب ، وتمكنت آن من الهرب على متن قارب بصعوبة. وعندما توفي رئيس أساقفة كانتربري ويليام وارهام عام 1532 ، تم تعيين قسيس عائلة بولين توماس كرانمر بموافقة بابوية. نفوذ روما في إنجلترا. ففي عام 1532 ، قدم توماس كرومويل أمام البرلمان من القوانين ، التي تعطي الملك حق'
Ground Truth: 'الملكة كاثرين.'
Match: ❌
--------------------------------------------------
Example 2:
Question: يحتاج زواج آمي وهنري إلى دعم أي ملك؟
Context snippet: خلال هذه الفترة، لعبت آن بولين دورًا بارزًا في علاقات إنجلترا الدولية، من خلال توطيد

In [11]:
from collections import Counter
import re

def normalize_arabic_text(text):
    """Normalize Arabic text for comparison"""
    # Remove diacritics and extra spaces
    text = re.sub(r'[\u064B-\u065F\u0670\u0640]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

def calculate_token_f1(predicted, ground_truth):
    """Calculate F1 score based on token overlap"""
    pred_tokens = normalize_arabic_text(predicted).split()
    truth_tokens = normalize_arabic_text(ground_truth).split()

    if len(pred_tokens) == 0 and len(truth_tokens) == 0:
        return 1.0

    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return 0.0

    common_tokens = Counter(pred_tokens) & Counter(truth_tokens)
    num_common = sum(common_tokens.values())

    if num_common == 0:
        return 0.0

    precision = num_common / len(pred_tokens)
    recall = num_common / len(truth_tokens)

    return 2 * (precision * recall) / (precision + recall)

def evaluate_model_subset(test_dataset, num_samples=100, model_type="BEFORE"):
    """Evaluate on a subset of the dataset with token F1"""
    
    total_samples = min(num_samples, len(test_dataset))
    print(f"🔍 Evaluating Arabic BERT on ARCD Dataset - {model_type} TRAINING")
    print("=" * 70)
    print(f"Evaluating on {total_samples} samples from ARCD dataset")
    print("=" * 70)

    f1_scores = []
    empty_predictions = 0
    exact_matches = 0

    print("Processing samples...")

    for i in range(total_samples):
        example = test_dataset[i]
        context = example['context']
        question = example['question']
        ground_truth = example['answers']['text'][0]

        predicted = answer_question(question, context)

        # Check for empty predictions
        if not predicted.strip() or predicted.strip() == "غير محدد":
            empty_predictions += 1
            f1_scores.append(0.0)
        else:
            f1 = calculate_token_f1(predicted, ground_truth)
            f1_scores.append(f1)
            
            # Check exact match (case-insensitive)
            if normalize_arabic_text(predicted) == normalize_arabic_text(ground_truth):
                exact_matches += 1

        # Show progress every 20 samples
        if (i + 1) % 20 == 0:
            print(f"Processed {i + 1}/{total_samples} samples...")

    # Calculate metrics
    avg_f1 = sum(f1_scores) / len(f1_scores)
    exact_match_score = exact_matches / total_samples

    print(f"\n📊 {model_type} TRAINING RESULTS:")
    print(f"🎯 Average Token F1: {avg_f1:.3f}")
    print(f"✅ Exact Match Score: {exact_match_score:.3f}")
    print(f"📊 Total Samples: {total_samples}")
    print(f"⚠️  Empty Predictions: {empty_predictions}")
    print(f"📈 Valid Predictions: {total_samples - empty_predictions}")

    # Additional stats
    non_zero_f1s = [f1 for f1 in f1_scores if f1 > 0]
    if non_zero_f1s:
        avg_non_zero_f1 = sum(non_zero_f1s) / len(non_zero_f1s)
        print(f"📊 Average F1 (excluding empty): {avg_non_zero_f1:.3f}")
    
    return avg_f1, exact_match_score

# Run evaluation BEFORE training
print("\n🔴 DETAILED EVALUATION BEFORE TRAINING")
baseline_f1, baseline_em = evaluate_model_subset(dataset, num_samples=100, model_type="BEFORE")

print(f"\n🔴 BASELINE METRICS (BEFORE TRAINING):")
print(f"Token F1: {baseline_f1:.3f}")
print(f"Exact Match: {baseline_em:.3f}")


🔴 DETAILED EVALUATION BEFORE TRAINING
🔍 Evaluating Arabic BERT on ARCD Dataset - BEFORE TRAINING
Evaluating on 100 samples from ARCD dataset
Processing samples...
Processed 20/100 samples...
Processed 40/100 samples...
Processed 60/100 samples...
Processed 80/100 samples...
Processed 100/100 samples...

📊 BEFORE TRAINING RESULTS:
🎯 Average Token F1: 0.024
✅ Exact Match Score: 0.000
📊 Total Samples: 100
⚠️  Empty Predictions: 0
📈 Valid Predictions: 100
📊 Average F1 (excluding empty): 0.088

🔴 BASELINE METRICS (BEFORE TRAINING):
Token F1: 0.024
Exact Match: 0.000


**4-Prepare Dataset for Training**

In [12]:
def prepare_training_features(examples):
    """Tokenize questions and contexts from ARCD dataset"""

    # Tokenize the question and context
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(tokenized_examples["offset_mapping"]):
        # Map back to original example index
        if "overflow_to_sample_mapping" in tokenized_examples:
            sample_index = tokenized_examples["overflow_to_sample_mapping"][i]
        else:
            sample_index = i
        sample_index = min(sample_index, len(answers) - 1)  # Bounds check

        # Get answer details
        current_answer = answers[sample_index]
        answer_start = current_answer["answer_start"][0]
        answer_text = current_answer["text"][0]
        answer_end = answer_start + len(answer_text)

        # Find token positions
        token_start_index = 0
        token_end_index = 0

        for idx, (start, end) in enumerate(offsets):
            if start is not None and end is not None:
                if start <= answer_start < end:
                    token_start_index = idx
                if start < answer_end <= end:
                    token_end_index = idx
                    break

        start_positions.append(token_start_index)
        end_positions.append(token_end_index)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    
    # Remove offset_mapping and overflow_to_sample_mapping to prevent training errors
    del tokenized_examples["offset_mapping"]
    if "overflow_to_sample_mapping" in tokenized_examples:
        del tokenized_examples["overflow_to_sample_mapping"]

    return tokenized_examples

# Split dataset (80% train, 20% eval)
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)

train_dataset = dataset.select(range(train_size))
eval_dataset = dataset.select(range(train_size, dataset_size))

# Apply preprocessing
tokenized_train = train_dataset.map(
    prepare_training_features,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_eval = eval_dataset.map(
    prepare_training_features,
    batched=True,
    remove_columns=eval_dataset.column_names
)

print(f"Training samples: {len(tokenized_train)}")
print(f"Evaluation samples: {len(tokenized_eval)}")
print(f"Training dataset columns: {tokenized_train.column_names}")

Map:   0%|          | 0/4268 [00:00<?, ? examples/s]

Map:   0%|          | 0/1067 [00:00<?, ? examples/s]

Training samples: 5477
Evaluation samples: 1308
Training dataset columns: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions']


**5-Define Training Arguments**

In [19]:
# Set up training arguments for Arabic BERT QA on ARCD dataset
training_args = TrainingArguments(
    output_dir="./arabic-bert-qa",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=200,
    eval_strategy="steps",
    eval_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    push_to_hub=False,
    warmup_steps=100,
)

**6- Train the Model**

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
)

/tmp/ipykernel_115975/3279074610.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [21]:
trainer.train()

Step,Training Loss,Validation Loss
200,3.196500,3.557044
400,3.409700,3.203139
600,3.168700,2.922780
800,3.102400,2.769380
1000,3.050200,2.731339
1200,2.910100,2.645916
1400,2.301800,2.585033
1600,2.032600,2.683618
1800,2.139500,2.645313
2000,1.718800,2.664501


TrainOutput(global_step=5480, training_loss=1.699539015414941, metrics={'train_runtime': 640.1946, 'train_samples_per_second': 34.221, 'train_steps_per_second': 8.56, 'total_flos': 4293367009929216.0, 'train_loss': 1.699539015414941, 'epoch': 4.0})

**7-Test AFTER Training (Fine-tuned Performance)**

In [22]:
# Test with real examples from the ARCD dataset - AFTER TRAINING
print("🟢 TESTING AFTER TRAINING (Fine-tuned Performance)")
print("=" * 60)
print("NOTE: Much better performance expected after fine-tuning!")
print("=" * 60)

# Get a few test examples from a different range to avoid train/eval overlap
test_examples = dataset.select(range(30, 35))  # Use samples 30-35 for testing

correct_predictions = 0
total_predictions = 0

for i, example in enumerate(test_examples):
    context = example['context']
    question = example['question']

    # Get ground truth answer
    ground_truth = example['answers']['text'][0]

    # Get model prediction
    predicted_answer = answer_question(question, context)

    # Check if prediction matches ground truth (case-insensitive)
    is_correct = predicted_answer.strip().lower() == ground_truth.strip().lower()
    if is_correct:
        correct_predictions += 1
    total_predictions += 1

    print(f"Example {i+1}:")
    print(f"Question: {question}")
    print(f"Context snippet: {context[:150]}...")
    print(f"Predicted: '{predicted_answer}'")
    print(f"Ground Truth: '{ground_truth}'")
    print(f"Match: {'✅' if is_correct else '❌'}")
    print("-" * 50)

# Calculate and display accuracy
trained_accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\n🟢 POST-TRAINING Accuracy: {correct_predictions}/{total_predictions} ({trained_accuracy:.1f}%)")

# Compare with baseline
print(f"\n📈 IMPROVEMENT:")
print(f"Before Training: {baseline_accuracy:.1f}%")
print(f"After Training: {trained_accuracy:.1f}%")
print(f"Improvement: +{trained_accuracy - baseline_accuracy:.1f} percentage points")

🟢 TESTING AFTER TRAINING (Fine-tuned Performance)
NOTE: Much better performance expected after fine-tuning!
Example 1:
Question: من دعم الرأي العام؟
Context snippet: ومع ذلك، ظل العامة يتعاطفون مع الملكة كاثرين. وفي ليلة من خريف عام 1531، بينما كانت آن تتناول طعامها في منزل ريفي على نهر التايمز، هاجمها حشد نسائي غا...
Predicted: 'كاثرين'
Ground Truth: 'الملكة كاثرين.'
Match: ❌
--------------------------------------------------
Example 2:
Question: يحتاج زواج آمي وهنري إلى دعم أي ملك؟
Context snippet: خلال هذه الفترة، لعبت آن بولين دورًا بارزًا في علاقات إنجلترا الدولية، من خلال توطيد التحالف مع فرنسا. فقد وطدت صلات جيدة مع السفير الفرنسي "جيل دي لا...
Predicted: 'فرانسوا الأول'
Ground Truth: 'الفرنسي'
Match: ❌
--------------------------------------------------
Example 3:
Question: كم كان مقدار المعاش السنوي؟
Context snippet: كما استفادت عائلة آن من هذه العلاقة، فأصبح والدها إيرلاً لويلتشير، وإتفق هنري أيضًا مع ابن عم آن الأيرلندي، ونصبه إيرلاً لأرموند. وفي مأدبة الاحتفال ب...
Predicted

In [23]:
# Run evaluation AFTER training
print("\n🟢 DETAILED EVALUATION AFTER TRAINING")
trained_f1, trained_em = evaluate_model_subset(dataset, num_samples=100, model_type="AFTER")

print(f"\n🟢 TRAINED MODEL METRICS (AFTER TRAINING):")
print(f"Token F1: {trained_f1:.3f}")
print(f"Exact Match: {trained_em:.3f}")

print(f"\n📊 COMPLETE COMPARISON:")
print("=" * 50)
print(f"{'Metric':<20} {'Before':<10} {'After':<10} {'Improvement':<15}")
print("=" * 50)
print(f"{'Token F1':<20} {baseline_f1:<10.3f} {trained_f1:<10.3f} {'+' + str(round(trained_f1 - baseline_f1, 3)):<15}")
print(f"{'Exact Match':<20} {baseline_em:<10.3f} {trained_em:<10.3f} {'+' + str(round(trained_em - baseline_em, 3)):<15}")
print("=" * 50)

# Performance interpretation
if trained_f1 >= 0.8:
    print("🎉 EXCELLENT performance achieved after fine-tuning!")
elif trained_f1 >= 0.6:
    print("✅ GOOD performance achieved after fine-tuning!")
elif trained_f1 >= 0.4:
    print("⚠️ FAIR performance achieved after fine-tuning!")
else:
    print("❌ Poor performance - consider longer training or hyperparameter tuning")


🟢 DETAILED EVALUATION AFTER TRAINING
🔍 Evaluating Arabic BERT on ARCD Dataset - AFTER TRAINING
Evaluating on 100 samples from ARCD dataset
Processing samples...
Processed 20/100 samples...
Processed 40/100 samples...
Processed 60/100 samples...
Processed 80/100 samples...
Processed 100/100 samples...

📊 AFTER TRAINING RESULTS:
🎯 Average Token F1: 0.413
✅ Exact Match Score: 0.230
📊 Total Samples: 100
⚠️  Empty Predictions: 4
📈 Valid Predictions: 96
📊 Average F1 (excluding empty): 0.608

🟢 TRAINED MODEL METRICS (AFTER TRAINING):
Token F1: 0.413
Exact Match: 0.230

📊 COMPLETE COMPARISON:
Metric               Before     After      Improvement    
Token F1             0.024      0.413      +0.389         
Exact Match          0.000      0.230      +0.23          
⚠️ FAIR performance achieved after fine-tuning!


**8-Advanced Inference with Confidence Scores**

In [24]:
def arabic_qa_inference(question, context, show_confidence=True):
    """
    Perform inference using your trained Arabic BERT model
    """
    # Tokenize input
    inputs = tokenizer(
        question,
        context,
        return_tensors="pt",
        max_length=512,
        truncation=True,
        padding=True
    )

    # Move to device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model.to(device)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get start and end logits
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Get probabilities for confidence score
    start_probs = torch.softmax(start_logits, dim=1)
    end_probs = torch.softmax(end_logits, dim=1)

    # Find the most likely start and end positions
    start_index = torch.argmax(start_logits, dim=1).item()
    end_index = torch.argmax(end_logits, dim=1).item()

    # Ensure end_index is not before start_index
    if end_index < start_index:
        end_index = start_index

    # Calculate confidence
    confidence = (start_probs[0][start_index] * end_probs[0][end_index]).item()

    # Extract answer tokens
    answer_tokens = inputs["input_ids"][0][start_index:end_index+1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    # Handle empty answers
    if not answer.strip():
        answer = "غير محدد"
        confidence = 0.0

    if show_confidence:
        return answer.strip(), confidence
    else:
        return answer.strip()

# Test inference with custom Arabic examples
print("🤖 Arabic BERT QA Inference System (TRAINED MODEL)")
print("=" * 60)

# Example 1: Technology
context1 = """
الذكاء الاصطناعي هو محاكاة الذكاء البشري في الآلات المبرمجة للتفكير والتعلم.
يشمل تطبيقات مثل التعرف على الكلام، والتعلم الآلي، والتخطيط، وحل المشكلات.
تستخدم شركات التكنولوجيا الذكاء الاصطناعي في منتجاتها لتحسين تجربة المستخدم.
"""

question1 = "ما هو الذكاء الاصطناعي؟"
answer1, conf1 = arabic_qa_inference(question1, context1)

print(f"Question: {question1}")
print(f"Answer: {answer1}")
print(f"Confidence: {conf1:.3f}")
print("-" * 30)

# Example 2: History
context2 = """
تأسست المملكة العربية السعودية على يد الملك عبد العزيز آل سعود في عام 1932.
تعتبر الرياض العاصمة وأكبر المدن، وتقع في وسط شبه الجزيرة العربية.
اللغة الرسمية هي العربية والعملة هي الريال السعودي.
"""

question2 = "متى تأسست المملكة العربية السعودية؟"
answer2, conf2 = arabic_qa_inference(question2, context2)

print(f"Question: {question2}")
print(f"Answer: {answer2}")
print(f"Confidence: {conf2:.3f}")
print("-" * 30)

# Example 3: Science
context3 = """
الجاذبية هي قوة طبيعية تجذب الأجسام نحو مركز الأرض. اكتشف إسحاق نيوتن قانون الجاذبية العام.
تبلغ سرعة الضوء في الفراغ 299,792,458 متر في الثانية، وهي أسرع سرعة ممكنة في الكون.
"""

question3 = "من اكتشف قانون الجاذبية؟"
answer3, conf3 = arabic_qa_inference(question3, context3)

print(f"Question: {question3}")
print(f"Answer: {answer3}")
print(f"Confidence: {conf3:.3f}")
print("-" * 30)

# Example 4: Geography
context4 = """
جبل إيفرست هو أعلى جبل في العالم، ويبلغ ارتفاعه 8,848.86 متر فوق مستوى سطح البحر.
يقع في جبال الهيمالايا على الحدود بين نيبال والتبت. تم الوصول إلى القمة لأول مرة عام 1953.
"""

question4 = "ما ارتفاع جبل إيفرست؟"
answer4, conf4 = arabic_qa_inference(question4, context4)

print(f"Question: {question4}")
print(f"Answer: {answer4}")
print(f"Confidence: {conf4:.3f}")
print("=" * 60)

print(f"\n🎯 SUMMARY:")
print(f"The Arabic model shows high confidence in its answers after fine-tuning!")
print(f"Average confidence: {(conf1 + conf2 + conf3 + conf4) / 4:.3f}")

🤖 Arabic BERT QA Inference System (TRAINED MODEL)
Question: ما هو الذكاء الاصطناعي؟
Answer: محاكاة الذكاء البشري في الآلات المبرمجة للتفكير والتعلم
Confidence: 0.201
------------------------------
Question: متى تأسست المملكة العربية السعودية؟
Answer: 1932.
Confidence: 0.527
------------------------------
Question: من اكتشف قانون الجاذبية؟
Answer: إسحاق نيوتن
Confidence: 0.588
------------------------------
Question: ما ارتفاع جبل إيفرست؟
Answer: 8, 848. 86 متر
Confidence: 0.179

🎯 SUMMARY:
The Arabic model shows high confidence in its answers after fine-tuning!
Average confidence: 0.373


Contributed by: Ali H